<a href="https://colab.research.google.com/github/apetrasc/Notebook-for-Bishop-Deep-Learning/blob/main/notebooks/12.transformers/Audiodata_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

"""
Created on Thur Sep 12 2024

@author: Sadanori Matsubara
"""
Ref: https://www.bishopbook.com/

Foundations of deep learning の12.4.3 Audio dataの解説を行います。音とは、空気の圧力変動として信号波形の形で保存されます。信号波形を直接的にDLモデルの入力として用いることは可能ではありますが、実用上は前処理としてMel spectrogramに通す、ということが一般的に用いられます。  
transformerは、特に音声分類のために用いられることが多いです。モデルの性能を評価するために、Audioset Datasetというデータセットがよく用いられます。これは、各音声波形に対し「車」「動物」「笑い声」といったラベルが割り振られており、何らかの音声波形を通すことによってそのクラスを予測するといったものになります。Transformer以前はmel spectrogramを画像として扱い、CNNの入力として利用するというアプローチがとられていました。Transformerの利点の一つとして長い時間での依存関係・相関をとらえることができるという特徴があり、これが音声データを扱う上で非常に重要になってきます。  
ここでは、mel spectrogramはまず画像として扱われ、トークンという一つの単位として分類されます。ViT(Vision Transformers)と同様の仕組みで画像をパッチに分割し、それらを一列に並べます。この場合、256の長さに並び替えられます。位置情報がこれらトークンに追加され、TransformerのEncoder層に通されます。最後のtransformer層からの出力トークンは、線形層によって複号され、ソフトマックス活性化関数を通すことによって出力が得られます。これらすべてのモデルは、クロスエントロピー誤差関数を用いることによってEnd-to-Endに訓練を行うことができます。  
以下は、これらの性能比較を行うためにCNNとTransformerモデルを実装したものです。注意として、これらの実験は再現不可能である可能性があります。ViTに関しては、1080Tiに1日程度計算させる（epoch=200）ことによって良いモデルを作成することができるという計算効率の良さは証明済ですが、本稿で紹介されているASTモデルに関しては超高性能なコンピュータを使用している可能性があり、再現の目途が立っていません。  
しかし、紹介されている論文(Gong,Chung,and Glass,2021)を見る限りアーキテクチャはViTに酷似しており、データの取り扱いも類似のものが見て取れましたのでViTでも同様の研究は再現可能だと考え実行を行っております。この訓練フェーズはAlvisという北欧でのリモートコンピューティングサービスを使用して、V100に計算を実行させています。

dataset:https://huggingface.co/datasets/agkphysics/AudioSet  
structure:

```
{
 'video_id': '--PJHxphWEs',
 'audio': {
  'path': 'audio/bal_train/--PJHxphWEs.flac',
  'array': array([-0.04364824, -0.05268681, -0.0568949 , ...,  0.11446512,
          0.14912748,  0.13409865]),
  'sampling_rate': 48000
 },
 'labels': ['/m/09x0r', '/t/dd00088'],
 'human_labels': ['Speech', 'Gush']
}

```
citation:


```
@inproceedings{jort_audioset_2017,
    title	= {Audio Set: An ontology and human-labeled dataset for audio events},
    author	= {Jort F. Gemmeke and Daniel P. W. Ellis and Dylan Freedman and Aren Jansen and Wade Lawrence and R. Channing Moore and Manoj Plakal and Marvin Ritter},
    year	= {2017},
    booktitle	= {Proc. IEEE ICASSP 2017},
    address	= {New Orleans, LA}
}
```



In [ ]:
# 利用可能なデータセットの一覧
!pip install --upgrade huggingface_hub
from huggingface_hub import hf_hub_download
import pandas as pd

REPO_ID = "YOUR_REPO_ID"
FILENAME = "data.csv"

dataset = pd.read_csv(
    hf_hub_download(repo_id=REPO_ID, filename=FILENAME, repo_type="dataset")
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-66e2eeeb-25908fc82f6d60f542a48826;599a2d2f-b484-4c48-ba43-a21f9a99cd55)

Repository Not Found for url: https://huggingface.co/datasets/YOUR_REPO_ID/resolve/main/data.csv.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.